In [1]:
import torch
#import nlp
from transformers import LongformerTokenizerFast

In [2]:
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096')

# Dataset processing

In [3]:
# with open("nq/train_wiki3_na_filtered_qg_t5l35-sqd_filtered.json", "r") as fh:
#     train_dataset = json.load(fh)

In [3]:
from datasets import load_dataset


dataset = load_dataset("natural_questions")

Using custom data configuration default
Reusing dataset natural_questions (/home/sri/.cache/huggingface/datasets/natural_questions/default/0.0.2/867dbbaf9137c1b83ecb19f5eb80559e1002ea26e702c6b919cfa81a17a8c531)


In [ ]:
example = dataset['train'][1]

start_idx, end_idx = example['annotations']['long_answer'][0]['start_token'], example['annotations']['long_answer'][0]['end_token']

" ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][start_idx:end_idx]) if not example['document']['tokens']['is_html'][start_idx+idx]])

In [ ]:
import jsonlines

INPUT_FILE = "natural-questions/simplified-nq-train.jsonl"
START_TOKEN = 3521
END_TOKEN = 3525
QAS_ID = 4549465242785278785
REMOVE_HTML = True


def get_span_from_token_offsets(start_token, end_token, qas_id,
                                remove_html):
    for obj in f:
        if obj["example_id"] != qas_id:
            continue
        print("Found")
        return obj
        if remove_html:
            answer_span = [
                item["token"]
                for item in obj["document_tokens"][start_token:end_token]
                if not item["html_token"]
            ]
        else:
            answer_span = [
                item["token"]
                for item in obj["document_tokens"][start_token:end_token]
            ]

        return " ".join(answer_span)


with jsonlines.open(INPUT_FILE) as f:
    result = get_span_from_token_offsets(f, START_TOKEN, END_TOKEN, QAS_ID,
                                         REMOVE_HTML)

#print(result)

In [ ]:
for example in dataset['train']:
    if example['id']!='5648415780048191748':
        start_idx, end_idx = example['annotations']['long_answer'][0]['start_token'], example['annotations']['long_answer'][0]['end_token']
        answer  = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][start_idx:end_idx]) if not example['document']['tokens']['is_html'][start_idx+idx]])
        context = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][:]) if not example['document']['tokens']['is_html'][idx]])
        query = example['question']['text']
        input_pairs =[query, context]
        encodings = tokenizer.encode_plus(input_pairs, pad_to_max_length=True, max_length=4096)
        context_encodings = tokenizer.encode_plus(context)

        start_byte_idx = context.find(answer)
        end_byte_idx = start_byte_idx + len(answer)-1
        #print(start_idx)
        start_positions_context = context_encodings.char_to_token(start_byte_idx)
        end_positions_context = context_encodings.char_to_token(end_byte_idx)

        sep_idx = encodings['input_ids'].index(tokenizer.sep_token_id)
        print(len(encodings['input_ids']))
        #print(context,"\n\n" ,answer)
        start_positions = start_positions_context + sep_idx + 1
        end_positions = end_positions_context + sep_idx + 1

        if end_positions > 4096:
              start_positions, end_positions = 0, 0

                
        encodings.update({'start_positions': start_positions,
                              'end_positions': end_positions,
                              'attention_mask': encodings['attention_mask']})
        #break

In [ ]:
context[start_byte_idx:end_byte_idx]

In [ ]:
end_byte_idx - start_byte_idx

In [4]:
def convert_to_features(example, max_length=4096):
    try:
    #example = dataset['train'][0]
        start_idx, end_idx = example['annotations']['long_answer'][0]['start_token'], example['annotations']['long_answer'][0]['end_token']
        answer  = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][start_idx:end_idx]) if not example['document']['tokens']['is_html'][start_idx+idx]])
        context = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][:]) if not example['document']['tokens']['is_html'][idx]])
        query = example['question']['text']
        input_pairs =[query, context]
        #encodings = tokenizer.encode_plus(input_pairs, pad_to_max_length=True, truncation=True, max_length=max_length)
        encodings = tokenizer.encode_plus(input_pairs, padding='max_length', truncation=True, max_length=max_length)
        context_encodings = tokenizer.encode_plus(context)

        start_byte_idx = context.find(answer)
        end_byte_idx = start_byte_idx + len(answer)-1

        if start_idx==-1:
            #print(start_idx)
            encodings.update({'start_positions': 0,
                              'end_positions': 0,
                              'attention_mask': encodings['attention_mask']})
            return encodings
        start_positions_context = context_encodings.char_to_token(start_byte_idx)
        end_positions_context = context_encodings.char_to_token(end_byte_idx)

        sep_idx = encodings['input_ids'].index(tokenizer.sep_token_id)

        start_positions = start_positions_context + sep_idx + 1
        end_positions = end_positions_context + sep_idx + 1

        if end_positions > 4096:
              start_positions, end_positions = 0, 0

        encodings.update({'start_positions': start_positions,
                              'end_positions': end_positions,
                              'attention_mask': encodings['attention_mask']})
    except Exception as e:
        #print(start_idx, example)
        print(e, example['id'])
    return encodings

In [6]:
example = dataset['train'][0]
start_idx, end_idx = example['annotations']['long_answer'][0]['start_token'], example['annotations']['long_answer'][0]['end_token']
answer  = " ".join([itm for idx, itm in enumerate(example['document']['tokens']['token'][start_idx:end_idx]) if not example['document']['tokens']['is_html'][start_idx+idx]])

In [9]:
convert_to_features(example).keys()

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])

In [10]:
train_dataset = dataset['train'].select(indices=list(range(32)))

In [5]:
train_dataset = dataset['train']
train_dataset = train_dataset.map(convert_to_features)

torch.save(train_dataset, 'data/train_data_full_4096.pt')

Token indices sequence length is longer than the specified maximum sequence length for this model (6404 > 4096). Running this sequence through the model will result in indexing errors


  0%|          | 0/307373 [00:00<?, ?ex/s]

In [6]:
columns = ['input_ids', 'attention_mask', 'start_positions', 'end_positions']
train_dataset.set_format(type='torch', columns=columns)

torch.save(train_dataset, 'data/train_data_4096.pt')

In [8]:
#valid_dataset = dataset['validation'].select(indices=list(range(32)))
valid_dataset = dataset['validation']
valid_dataset = valid_dataset.map(convert_to_features, load_from_cache_file=False)

torch.save(valid_dataset, 'data/valid_data_full_4096.pt')

columns = ['input_ids', 'attention_mask', 'start_positions', 'end_positions']
valid_dataset.set_format(type='torch', columns=columns)

torch.save(valid_dataset, 'data/valid_data_4096.pt')

  0%|          | 0/7830 [00:00<?, ?ex/s]

In [ ]:
torch.save(train_dataset, 'data/train_data_full.pt')
#torch.save(valid_dataset, 'data/valid_data_full.pt')

In [ ]:
columns = ['input_ids', 'attention_mask', 'start_positions', 'end_positions']
train_dataset.set_format(type='torch', columns=columns)
#valid_dataset.set_format(type='torch', columns=columns)

In [ ]:
len(train_dataset), len(valid_dataset)

In [ ]:
example['annotations']['long_answer'][0]

In [ ]:
# cach the dataset, so we can load it directly for training

torch.save(train_dataset, 'data/train_data.pt')
#torch.save(valid_dataset, 'data/valid_data.pt')

In [ ]:
# train_dataset = torch.load("data/train_data.pt")
# valid_dataset = torch.load("data/valid_data.pt")

# Training Script

In [ ]:
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, List, Optional

import numpy as np
import torch

from transformers import LongformerForQuestionAnswering, LongformerTokenizerFast, EvalPrediction
from transformers import (
    HfArgumentParser,
    DataCollator,
    Trainer,
    TrainingArguments,
    set_seed,
)

logger = logging.getLogger(__name__)

In [ ]:
@dataclass
class ModelArguments:
    """
    model/config/tokenizer arguments
    """
    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from hugginface.co/models"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )
    

@dataclass
class DataTrainingArguments:
    """
    model input arguments
    """
    train_file_path: Optional[str] = field(
        default='data/train_data.pt',
        metadata={"help": "Path for cached train dataset"}
    )
    valid_file_path: Optional[str] = field(
        default='data/valid_data.pt',
        metadata={"help": "Path for cached valid dataset"}
    )
    max_len: Optional[str] = field(
        default=4096,
        metadata={"help": "Max input length for the source text"}
    )
        

class DummyDataCollator():
    def __call__(self, batch: List) -> Dict[str, torch.Tensor]:
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
            A dictionary of tensors
        """
        input_ids = torch.stack([example['input_ids'] for example in batch])
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        start_positions = torch.stack([example['start_positions'] for example in batch])
        end_positions = torch.stack([example['end_positions'] for example in batch])

        return {
            'input_ids': input_ids, 
            'start_positions': start_positions, 
            'end_positions': end_positions,
            'attention_mask': attention_mask
        }

In [ ]:
def main():
    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    
    model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath('args.json'))
    
    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )
        
    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        training_args.local_rank,
        training_args.device,
        training_args.n_gpu,
        bool(training_args.local_rank != -1),
        training_args.fp16,
    )
    logger.info("Training/evaluation parameters %s", training_args)

    # Set seed
    set_seed(training_args.seed)

    # Load pretrained model and tokenizer
    #
    # Distributed training:
    # The .from_pretrained methods guarantee that only one local process can concurrently
    # download model & vocab.
    
    tokenizer = LongformerTokenizerFast.from_pretrained(
        model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    model = LongformerForQuestionAnswering.from_pretrained(
        model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
    
    # Get datasets
    print('loading data')
    train_dataset  = torch.load(data_args.train_file_path)
    valid_dataset = torch.load(data_args.valid_file_path)
    print('loading done')
    
    
    # Initialize our Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=DummyDataCollator(),
 #       prediction_loss_only=True,
    )
    
    # Training
    if training_args.do_train:
        trainer.train(
            model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
        )
        trainer.save_model()
        # For convenience, we also re-save the tokenizer to the same directory,
        # so that you can share your model easily on huggingface.co/models =)
        if trainer.is_world_master():
            tokenizer.save_pretrained(training_args.output_dir)

    # Evaluation
    results = {}
    if training_args.do_eval and training_args.local_rank in [-1, 0]:
        logger.info("*** Evaluate ***")

        eval_output = trainer.evaluate()

        output_eval_file = os.path.join(training_args.output_dir, "eval_results.txt")
        with open(output_eval_file, "w") as writer:
            logger.info("***** Eval results *****")
            for key in sorted(eval_output.keys()):
                logger.info("  %s = %s", key, str(eval_output[key]))
                writer.write("%s = %s\n" % (key, str(eval_output[key])))
    
        results.update(eval_output)
    
    return results

def _mp_fn(index):
    # For xla_spawn (TPUs)
    main()

In [ ]:
import json

args_dict = {
  "n_gpu": 2,
  "model_name_or_path": 'allenai/longformer-base-4096',
  "max_len": 4096 ,
  "output_dir": './models',
  "overwrite_output_dir": True,
  "per_gpu_train_batch_size": 8,
  "per_gpu_eval_batch_size": 8,
  "gradient_accumulation_steps": 16,
  "learning_rate": 1e-4,
  "num_train_epochs": 3,
  "do_train": True
}

with open('args.json', 'w') as f:
    json.dump(args_dict, f)

In [ ]:
main()

In [ ]:
for d in  train_dataset:
    print (d['input_ids'].shape)

In [ ]:
dataset

# args testing

In [10]:
import torch
from transformers import LongformerTokenizerFast, LongformerForQuestionAnswering
from tqdm.auto import tqdm

In [11]:
#file_path="save/train/longFormer-optimal-search/"
file_path="valhalla/longformer-base-4096-finetuned-squadv1"
tokenizer = LongformerTokenizerFast.from_pretrained(file_path)
model = LongformerForQuestionAnswering.from_pretrained(file_path)
model = model.cuda()
model.eval()

LongformerForQuestionAnswering(
  (longformer): LongformerModel(
    (embeddings): LongformerEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(4098, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LongformerEncoder(
      (layer): ModuleList(
        (0): LongformerLayer(
          (attention): LongformerAttention(
            (self): LongformerSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (query_global): Linear(in_features=768, out_features=768, bias=True)
              (key_global): Linear(in_features=768, out_features=768, bias=True)
              (value_gl

In [12]:
valid_dataset = torch.load('data/valid_data_opt.pt')
dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=1)

In [31]:
answers = []
with torch.no_grad():
    for batch in tqdm(dataloader):
        #if batch['end_positions']!=0:
        #start_scores, end_scores = model(input_ids=batch['input_ids'].cuda(),
        #                          attention_mask=batch['attention_mask'].cuda())
        outputs = model(input_ids=batch['input_ids'].cuda(),
                                  attention_mask=batch['attention_mask'].cuda())
        for i in range(outputs['start_logits'].shape[0]):
            all_tokens = tokenizer.convert_ids_to_tokens(batch['input_ids'][i])
            answer = ' '.join(all_tokens[torch.argmax(outputs['start_logits'][i]) : torch.argmax(outputs['end_logits'][i])+1])
            ans_ids = tokenizer.convert_tokens_to_ids(answer.split())
            answer = tokenizer.decode(ans_ids)
            answers.append(answer)
            #print (answer, "\n\n")

  0%|          | 0/32 [00:00<?, ?it/s]

In [34]:
def postprocess(token):
        with_space = False
        with_break = False
        if token.startswith('Ġ'):
            with_space = True
            token = token[1:]
        elif token.startswith('â'):
            token = ' '
        elif token.startswith('Ċ'):
            token = ' '
            with_break = True

        token = '-' if token.startswith('â') else token
        token = '“' if token.startswith('ľ') else token
        token = '”' if token.startswith('Ŀ') else token
        token = "'" if token.startswith('Ļ') else token

        return token
    
predictions = []
references = []
for ref, pred in zip(valid_dataset, answers):
    predictions.append(pred)
    all_tokens = tokenizer.convert_ids_to_tokens(ref['input_ids'])
    correct_answer = all_tokens[valid_dataset[4]['start_positions']:valid_dataset[4]['end_positions']]
    correct_answer = [postprocess(i) for i in correct_answer]
    references.append(correct_answer)

In [27]:
all_tokens = tokenizer.convert_ids_to_tokens(valid_dataset[4]['input_ids'])
#answer = ' '.join(all_tokens[tokenizer.convert_ids_to_tokens(:])

In [35]:
## SQuAD evaluation script. Modifed slightly for this notebook

from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(gold_answers, predictions):
    f1 = exact_match = total = 0

    for ground_truths, prediction in zip(gold_answers, predictions):
        total += 1
        exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(
          f1_score, prediction, ground_truths)
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

In [36]:
evaluate(references, predictions)

{'exact_match': 9.375, 'f1': 9.8783239124184}

In [21]:
valid_dataset['input_ids'][valid_dataset[4]['start_positions']:valid_dataset[4]['end_positions']]

tensor([[    0, 12196,  3508,  ...,  7964,    31,     2],
        [    0,  8569,   222,  ..., 15994,    69,     2],
        [    0,  8155,   300,  ...,   518,     7,     2]])

In [21]:
all_tokens = tokenizer.convert_ids_to_tokens(batch['input_ids'][0])
answer = ' '.join(all_tokens[torch.argmax(outputs['start_logits']) : torch.argmax(outputs['end_logits'])+1])
ans_ids = tokenizer.convert_tokens_to_ids(answer.split())
answer = tokenizer.decode(ans_ids)
answers.append(answer)

In [22]:
answers

['<s>']

In [27]:
batch

{'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]]),
 'end_positions': tensor([0]),
 'input_ids': tensor([[   0, 8155,   34,  ..., 2321, 2321,    2]]),
 'start_positions': tensor([0])}